<a href="https://colab.research.google.com/github/mauriciodev/spatialgeodesy/blob/main/exercises/Ex3_GNSS_observations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 3 - GNSS Observations

In this exercise we are going to download two observation RINEX files (versions 2 and 3) from IBGE's RBMC (Rede Brasileira de Monitoramento Contínuo), open with the Georinex library and plot the differences between the pseudodistance and carrier phase.
In this exercise you're going to use the RINEX version 2 code as example and write the version 3 code.

# Preparation

## Installing dependencies

In [ ]:
#Python standard libraries
import os #File path operations.
import shutil #Shell operations. Unzipping, moving files, etc.
import urllib.request #Downloader.

#External libraries
import numpy as np #Numeric Python.
import pandas as pd #Python Data Analysis Library.
import matplotlib.pyplot as plt #Plots.
import xarray as xa #Multi dimension arrays. For georinex.

In [ ]:
# Installing the package that reads rinex and sp3 files
!pip install georinex
# The speed for loading RINEX 3 files on current georinex is terrible.
import georinex

# Input parameters

Go to http://www.bdg.ibge.gov.br/appbdg/ , turn on RBMC in "Camadas" and choose one station. Change the value of the "station" variable for your chosen station.

In [ ]:
#This is the reference time. We are going to use it to download the files.
t = pd.to_datetime('2023-01-01 02:00:01')
station="RJNI" #This is the station.

In [ ]:
year = t.year
dayOfYear = t.day_of_year
twoDigitYear = t.year %100

#File name is based on the Day of Year
#formatiing with previous variables. Note the :03 on each variable to fill with leading zeros.
url_v2=f"https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/dados/{year}/{dayOfYear:03}/{station.lower()}{dayOfYear:03}1.zip"
print("URL for the version 2 RINEX file:", url_v2)

Please go to https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/ to understand how the RBMC data is provided.

## Downloading the receiver's observation file

In [ ]:
#splitting the file name from the URL
obsRinexFile_v2=os.path.split(url_v2)[1]
urllib.request.urlretrieve(url_v2, obsRinexFile_v2) #download the file saving as the name in obsRinexFile_v2
print(obsRinexFile_v2)

# Rinex version 2 file

In [ ]:
# the ! prefix runs the commands on a linux shell. So we can use both python and shell commands on this notebook.
import shutil #Shell operations. Unzipping, moving files, etc.
shutil.unpack_archive(obsRinexFile_v2)
uncompressed=obsRinexFile_v2[:-3]

for f in os.listdir(): print(f) #listing files in the current directory

In [ ]:
obsFile_v2=obsRinexFile_v2[:-3]+f"{twoDigitYear}d"
print("Observation file: ", obsFile_v2)

## Showing the first 30 lines of the observation file

In [ ]:
with open(obsFile_v2) as f:
  #printing the first 30 lines of the rinex file.
  print(''.join(f.readlines()[:30]))
  #Note that f.readlines() returns a list.

Note that on RINEX 2 the file ending with "d" is the multignss data, but it doesn't have all the systems.
We can't read anything because it's in Hatanaka compressed RINEX format. Converting it to regular RINEX files require the hatanaka program, that is included with Georinex.

In [ ]:
#reading a single constellation is A LOT faster in Georinex
obsData=georinex.load(obsFile_v2, use='G')
obsData

Take your time to inspect the data above. \
The "coordinates" show that observations can be selected using sv (spatial vehicle) and time. \
The "Data variables" show that we have psudoranges (Ex.: C1, P1), carrier phase (Ex.: L1) and signal strength (Ex.: S1)
Note that because this is a multiGNSS file, there is a lot of nan (missing values). For example, GPS satellites only provide data on bands 1, 2 and 5, so C7 measures will be nan.

## Plotting GPS satellite 1 measures

Here we are going to filter the observation file and select only one vehicle.

In [ ]:
sv='G01'
obsData.sel(sv=sv).C1

In [ ]:
sv='G01'
plt.plot(obsData.sel(sv=sv).C1, label="Pseudorange Band 1")
plt.plot(obsData.sel(sv=sv).C2, label="Pseudorange Band 2")
plt.plot(obsData.sel(sv=sv).L1, label="Phase Band 1")
plt.plot(obsData.sel(sv=sv).L2, label="Phase Band 2")
plt.xlabel("Seconds of the day")
plt.ylabel("Distance measured")
plt.title(f"Vehicle: {sv}")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.plot()

In [ ]:
sv='G01'
plt.plot(obsData.sel(sv=sv).C1-obsData.sel(sv=sv).C2, label="Difference C1-C2")
plt.xlabel("Seconds of the day")
plt.ylabel("Difference (m)")
plt.title(f"Vehicle:{sv}")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.plot()

# **Rinex version 3 file (Please fill this part)**

In [ ]:
url_v3=f"https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/dados_RINEX3/{year}/{dayOfYear:03}/{station.upper()}00BRA_R_{year}{dayOfYear:03}0000_01D_15S_MO.crx.gz"
print("URL for the version 3 RINEX file:", url_v3)

In [ ]:
obsRinexFile_v3=os.path.split(url_v3)[1]
urllib.request.urlretrieve(url_v3, obsRinexFile_v3) #download the file saving as the name in obsRinexFile_v3
print(obsRinexFile_v3)

In [ ]:
#reading a single constellation is A LOT faster in Georinex, but still slow in RINEX 3
obsData=georinex.load(obsRinexFile_v3, use='G')
obsData

Plot the same graphs that we plotted with the Version 2 file.
Using: C1C, C2W, L1C, L2W